In [8]:
import requests
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, String, Date, Time

response = requests.get("https://api.jolpi.ca/ergast/f1/2024/8/results/")
race_data = response.json()

In [9]:
round_number = race_data.get('MRData', {}).get('RaceTable', {}).get('round', None)
race_name = race_data.get('MRData', {}).get('RaceTable', {}).get('Races', [{}])[0].get('raceName', None)
race_year = race_data.get('MRData', {}).get('RaceTable', {}).get('Races', [{}])[0].get('season', None)
race_date = race_data.get('MRData', {}).get('RaceTable', {}).get('Races', [{}])[0].get('date', None)

In [10]:
results = race_data.get('MRData', {}).get('RaceTable', {}).get('Races', [{}])[0].get('Results', None)

In [11]:
df = pd.DataFrame(results)

In [12]:
df['driver_id'] = df['Driver'].apply(lambda x: x['driverId'])
df['constructor_id'] = df['Constructor'].apply(lambda x: x['constructorId'])
df['driver'] = df['Driver'].apply(lambda x: f"{x['givenName']} {x['familyName']}")
df['constructor'] = df['Constructor'].apply(lambda x: x['name'])
df['time'] = df['Time'].apply(lambda x: x['time'] if isinstance(x, dict) else None)
df['FastestLap'] = 1 # for 2024 race 8 monaco data error
df['fastest_lap_rank'] = df['FastestLap'].apply(lambda x: x['rank'] if isinstance(x, dict) else None) 

In [13]:
df.insert(0, 'race_name', race_name)
df.insert(1, 'race_date', race_date)
df.insert(2, 'race_year', race_year)
df.insert(3, 'round_number', round_number)

In [14]:
df = df[['race_name', 'race_date', 'race_year', 'round_number', 
         'position', 'points', 'fastest_lap_rank', 'driver', 'driver_id', 
         'constructor', 'constructor_id', 'grid', 'status',]]

In [15]:
# PostgreSQL connection details
user = "f1_user"
password = "drivetosurvive"
host = "localhost"
port = "5433"
db = "drive_to_survive"

In [43]:
# Define SQL to drop and recreate the table
create_table_sql = """
DROP TABLE IF EXISTS results_2024;
CREATE TABLE results_2024 (
    race_name VARCHAR(255),
    race_date DATE,
    race_year INTEGER,
    round_number INTEGER,
    position INTEGER,
    points INTEGER,
    fastest_lap_rank INTEGER,
    driver VARCHAR(255),
    driver_id VARCHAR(255),
    constructor VARCHAR(255),
    constructor_id VARCHAR(255),
    grid INTEGER,
    status VARCHAR(255)
);
"""

In [16]:
# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [45]:
# Execute the table creation SQL
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()

In [17]:
dtype_mapping = {
    "race_name": String(255),
    "race_date": Date,
    "race_year": Integer,
    "round_number": Integer,
    "position": Integer,
    "points": Integer,
    "fastest_lap_rank": Integer,
    "driver": String(255),
    "driver_id": String(255),
    "constructor": String(255),
    "constructor_id": String(255),
    "grid": Integer,
    "status": String(255),
}

df.to_sql('results_2024', engine, if_exists='append', index=False, dtype=dtype_mapping)

20